In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F


time_embeds = [
            nn.Sequential(
                nn.Linear(1, 2),
                nn.SiLU(),
                nn.Linear(1, 2),
            )
            
        ] * 3

print(time_embeds)

[Sequential(
  (0): Linear(in_features=1, out_features=2, bias=True)
  (1): SiLU()
  (2): Linear(in_features=1, out_features=2, bias=True)
), Sequential(
  (0): Linear(in_features=1, out_features=2, bias=True)
  (1): SiLU()
  (2): Linear(in_features=1, out_features=2, bias=True)
), Sequential(
  (0): Linear(in_features=1, out_features=2, bias=True)
  (1): SiLU()
  (2): Linear(in_features=1, out_features=2, bias=True)
)]


In [28]:
import numpy as np
a = np.zeros(3)
b = np.ones(3)
c = b + 1

list_1 = [a,b,c]
list = [list_1] * 3
print(list)
list[0][1] += 2
print(list)

[[array([0., 0., 0.]), array([1., 1., 1.]), array([2., 2., 2.])], [array([0., 0., 0.]), array([1., 1., 1.]), array([2., 2., 2.])], [array([0., 0., 0.]), array([1., 1., 1.]), array([2., 2., 2.])]]
[[array([0., 0., 0.]), array([3., 3., 3.]), array([2., 2., 2.])], [array([0., 0., 0.]), array([3., 3., 3.]), array([2., 2., 2.])], [array([0., 0., 0.]), array([3., 3., 3.]), array([2., 2., 2.])]]


In [21]:
class Proj(nn.Module):
    '''
    Project input `[batch_size, in_channels, width, height]`
    to output `[batch_size, out_channels, width, height]`
    '''
    def __init__(self, in_channels, out_channels, channels):
        super().__init__()
        self.in_conv = nn.Conv2d(in_channels, channels, kernel_size=3, padding=1)
        self.convs = nn.Sequential(
            nn.Conv2d(channels, 4*channels, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.Conv2d(4*channels, channels, kernel_size=3, padding=1),
        )
        self.out_conv = nn.Conv2d(channels, out_channels, kernel_size=3, padding=1)
        self.skip_conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)

    def forward(self, x):
        '''
        x: `[batch_size, track_num, multitrack_in_channels, width, height]`

        returns:
        `[batch_size, drum_channels, width, height]`
        '''
        
        h = F.tanh(self.in_conv(x))
        h = h + self.convs(h)
        h = F.tanh(self.out_conv(h))
        h = F.tanh(h + self.skip_conv(x))
        return h
    
    


In [22]:

m_to_d_proj = Proj(
    in_channels=track_num*multitrack_in_channels,
    out_channels=drum_channels,
    channels=channels,
)
print(m_to_d_proj(m_t).shape)

torch.Size([2, 1, 32, 32])
